In [18]:
import sys
snorkel_dir = '/Users/kunling/Dropbox/decisionengines/snorkel/'
#snorkel_dir = '/Users/kunling/Dropbox/decisionengines/snorkel/treedlib'
os.environ['SNORKELHOME'] = '/Users/kunling/Dropbox/decisionengines/snorkel/'
sys.path.insert(0,snorkel_dir)

In [15]:


#%load_ext autoreload
#%autoreload 2

print session
print session.connection()
from sqlalchemy import MetaData
import contextlib
meta = MetaData()

with contextlib.closing(session.connection()) as con:
    trans = con.begin()
    for table in reversed(meta.sorted_tables):
        con.execute(table.delete())
    trans.commit()

In [19]:

%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

# Running a Sapcy corpus parser
from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser

corpus_parser = CorpusParser() # use corenlp parser, slow but accurate for NER


# spacy lang_model = en_core_web_md
#en_depent_web_md

#corpus_parser = CorpusParser(parser=Spacy(lang='en_core_web_sm')) # use spacy parser, fast but not accurate enough for NER

Could not kill CoreNLP server [42975] [Errno 3] No such process


In [20]:
# Parse the document and save into the sqlite server
# see the intro-turtorial 1

# Configure a doc pre-processer
from snorkel.parser import TextDocPreprocessor
doc_preprocessor = TextDocPreprocessor(path = 'data/samples', encoding = "utf-8")
print doc_preprocessor.path

corpus_parser.apply(doc_preprocessor)



data/samples
Clearing existing...
Running UDF...
parts: abs_sent_offset [0, 4, 12, 17, 21, 27, 32, 38, 46, 49, 54, 64, 67, 73, 77, 84, 87, 92, 94, 97, 101, 113, 122, 126, 130, 139, 141, 143, 145, 149, 158, 161, 165, 169, 174, 179, 182, 184]
abs_sent_offset 0
abs_sent_offset_end 185
parts: abs_sent_offset [188, 193, 205, 216, 226, 227, 228, 237, 238, 240, 243, 248, 252, 260, 265, 268, 272, 279, 281, 287, 294, 295, 296, 306, 307, 309, 313, 323, 328, 329, 330, 336, 337, 338]
abs_sent_offset 188
abs_sent_offset_end 339
parts: abs_sent_offset [339, 344, 353, 357, 362, 372, 375, 383, 388, 394, 398, 400, 405, 408, 412, 414, 417, 421, 429, 436, 440, 441, 453, 458, 459, 461, 466, 477, 479, 481, 490, 494, 496, 499, 504, 506, 510, 516, 523, 525, 530, 531, 543, 550, 551, 553, 558, 560, 564, 566, 568, 572, 576, 578, 581, 586, 588, 591, 593, 597, 602, 605, 607]
abs_sent_offset 339
abs_sent_offset_end 608
parts: abs_sent_offset [0, 5, 14, 18, 23, 33, 36, 44, 49, 55, 60, 63, 66, 70, 73, 76, 80, 88, 98

In [19]:
# Check if the document has beed loaded
from snorkel.models import Document, Sentence
for doc in session.query(Document).all():
    print doc.id
    #print doc.sentences

1
2
3


In [20]:
# Define Candidate Type
from snorkel.models import candidate_subclass, Document, Candidate
rel_contract = candidate_subclass('Contractor-Contractee', ['org1', 'org2'])

# Write the candidate extractor
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import OrganizationMatcher
ngrams         = Ngrams(n_max=7)
org_matcher = OrganizationMatcher()
cand_extractor = CandidateExtractor(rel_contract, [ngrams, ngrams], [org_matcher, org_matcher])

In [21]:
# Split the data into train, dev and test 
from snorkel.models import Document

docs = session.query(Document).order_by(Document.name).all()

train_sents = set()
dev_sents   = set()
test_sents  = set()

for i, doc in enumerate(docs):
    for s in doc.sentences:
        if i % 3 == 1:
            dev_sents.add(s)
        elif i % 3 == 2:
            test_sents.add(s)
        else:
            train_sents.add(s)
            
print len(train_sents)
print len(test_sents)
print len(dev_sents)

3
1
4


In [22]:
# Apply candidate extractors on all the sentences
# Save the candidates into the databse
# The rel_candidates will be splitted into train, dev, test 
for i, sents in enumerate([train_sents, dev_sents, test_sents]):
    cand_extractor.apply(sents, split=i)
    print("Number of candidates:", session.query(rel_contract).filter(rel_contract.split == i).count())
    


Clearing existing...
Running UDF...
[========================================] 100%

('Number of candidates:', 2)
Clearing existing...
Running UDF...
[========================================] 100%

('Number of candidates:', 2)
Clearing existing...
Running UDF...
[========================================] 100%

('Number of candidates:', 1)


In [23]:
# Define the labeling functions
import re
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)
def LF_between_and(c):
    #print get_left_tokens(c[0], window=2)
    if "between" in get_left_tokens(c[0], window=5):
        #print get_between_tokens(c)
        if "and" in get_between_tokens(c):
            return 1
    return 0

def LF_agreement(c):
    if "agreement" in get_left_tokens(c[0], window=30):
        return 1
    return 0
    


In [24]:
# Test the first labeling function: LF_between_and
labeled = []
for c in session.query(rel_contract).filter(rel_contract.split == 0).all():
    if LF_between_and(c) != 0:
        labeled.append(c)
print("Number labeled:", len(labeled))
for i in range(len(labeled)):
    print labeled[i].get_parent()
    #print labeled[i].labels
    print




('Number labeled:', 2)
Sentence(Document b5dfde83-dde0-4235-b749-4c2aaf8abd06,2,THIS PURCHASE AND SALE AGREEMENT is entered into this  23rd, March,  2000, by and between Google Inc (hereinafter Buyer), 1545 Charleston Rd, Mountain View, CA 94043, and Cisco Systems, Inc. (hereinafter Company), 3571 N 1st St, San Jose, CA 95134, USA, and here is it.)

Sentence(Document b5dfde83-dde0-4235-b749-4c2aaf8abd06,1,This Independent Contractor Agreement ("Agreement") is made and entered into by and between, Cisco System ("Contractor") and Microsoft Corp (“Client”).)



In [25]:
# Test the second labeling function: LF_agreement
labeled = []
for c in session.query(rel_contract).filter(rel_contract.split == 0).all():
    if LF_agreement(c) != 0:
        labeled.append(c)
print("Number labeled:", len(labeled))
for i in range(len(labeled)):
    print labeled[i].get_parent()
    print labeled[i]
    print




('Number labeled:', 2)
Sentence(Document b5dfde83-dde0-4235-b749-4c2aaf8abd06,2,THIS PURCHASE AND SALE AGREEMENT is entered into this  23rd, March,  2000, by and between Google Inc (hereinafter Buyer), 1545 Charleston Rd, Mountain View, CA 94043, and Cisco Systems, Inc. (hereinafter Company), 3571 N 1st St, San Jose, CA 95134, USA, and here is it.)
Contractor-Contractee(Span("Google Inc", sentence=6, chars=[90,99], words=[18,19]), Span("Cisco Systems, Inc.", sentence=6, chars=[171,189], words=[36,39]))

Sentence(Document b5dfde83-dde0-4235-b749-4c2aaf8abd06,1,This Independent Contractor Agreement ("Agreement") is made and entered into by and between, Cisco System ("Contractor") and Microsoft Corp (“Client”).)
Contractor-Contractee(Span("Cisco System", sentence=5, chars=[93,104], words=[18,19]), Span("Microsoft Corp", sentence=5, chars=[125,138], words=[26,27]))



In [26]:
LFs = [LF_between_and, LF_agreement]

# Apply the labelling functions
from snorkel.annotations import LabelAnnotator
import numpy as np
labeler = LabelAnnotator(lfs=LFs)

np.random.seed(1701)
L_train = labeler.apply(split=0)
# L_train is essentially the sparse matrix
# check the labeling results

for i in range(3):
    #print L_train.get_candidate(session, i)
    print L_train.get_candidate(session, i).get_parent()
    print L_train.get_candidate(session, i).org1.get_span(), " ", L_train.get_candidate(session, i).org2.get_span() 
    print L_train.get_candidate(session, i).labels
    print 
L_train.lf_stats(session)

Clearing existing...
Running UDF...
[========================================] 100%

Sentence(Document b5dfde83-dde0-4235-b749-4c2aaf8abd06,2,THIS PURCHASE AND SALE AGREEMENT is entered into this  23rd, March,  2000, by and between Google Inc (hereinafter Buyer), 1545 Charleston Rd, Mountain View, CA 94043, and Cisco Systems, Inc. (hereinafter Company), 3571 N 1st St, San Jose, CA 95134, USA, and here is it.)
Google Inc   Cisco Systems, Inc.
[Label (LF_between_and = 1), Label (LF_agreement = 1)]

Sentence(Document b5dfde83-dde0-4235-b749-4c2aaf8abd06,1,This Independent Contractor Agreement ("Agreement") is made and entered into by and between, Cisco System ("Contractor") and Microsoft Corp (“Client”).)
Cisco System   Microsoft Corp
[Label (LF_between_and = 1), Label (LF_agreement = 1)]



KeyError: 2

In [27]:
# Launch Brat
from snorkel.contrib.brat import BratAnnotator

brat = BratAnnotator(session, rel_contract, encoding='utf-8', annotator_name='brat')

# Initilize the brat program
# the brat would copy the training data (split = 0) frrom the sqlite server 
# to the folder "snorkel/snorkel/snorkel/contrib/brat/brat-v1.3_Crunchy_Frog/data/contract/train"
brat.init_collection("contract/train", split=0, overwrite=True)

brat.view("contract/train")

Launching BRAT server at http://localhost:8001 [pid=92760]...
Killing BRAT server [92693]...
Removed existing collection at 'contract/train'


In [31]:
#load all the candidates and labeling results from the server
# the candiates are labeled 
train_cands = session.query(rel_contract).filter(rel_contract.split == 0).order_by(rel_contract.id).all()
for c in train_cands:
    print c
    print c.org1.get_span(), " ", c.org2.get_span()
    print c.labels
    print '-------------'

Contractor-Contractee(Span("Google Inc", sentence=6, chars=[90,99], words=[18,19]), Span("Cisco Systems, Inc.", sentence=6, chars=[171,189], words=[36,39]))
Google Inc   Cisco Systems, Inc.
[Label (LF_between_and = 1), Label (LF_agreement = 1)]
-------------
Contractor-Contractee(Span("Cisco System", sentence=5, chars=[93,104], words=[18,19]), Span("Microsoft Corp", sentence=5, chars=[125,138], words=[26,27]))
Cisco System   Microsoft Corp
[Label (LF_between_and = 1), Label (LF_agreement = 1)]
-------------


In [32]:
documents = session.query(Document).all()
print documents
for doc in documents:
    #print doc.sentences
    print doc.id


[Document b5dfde83-dde0-4235-b749-4c2aaf8abd06, Document b745346e-16a3-46ba-8adc-92b1fa085c38, Document c5dbaa73-8076-4385-944c-1539a30152d7]
1
2
3


In [33]:
# import the gold labels from the brat files 
# match the relationship stored in the brat folder "contract/train" 
# with the training candidates in the current databse
# Note that the candidates exsit in "contract/train" but not exisit in the "train_cands"
# will not be imported to the databse
# 

brat.import_gold_labels(session, "contract/train", train_cands)


# Now we can load the brat_labels from the current database(session)
# Note that the annotator_name='brat' becasue when we we initilize the BratAnnotator
# the default annotator name will be 'brat'
from snorkel.annotations import load_gold_labels
L_gold_train = load_gold_labels(session, annotator_name='brat', split=0)
print repr(L_gold_train)
print L_gold_train




documents
{u'b5dfde83-dde0-4235-b749-4c2aaf8abd06': Document b5dfde83-dde0-4235-b749-4c2aaf8abd06}

annotations
{'b5dfde83-dde0-4235-b749-4c2aaf8abd06': {u'R1': (u'Contractor-Contractee', u'T1', u'T2'), u'R2': (u'Contractor-Contractee', u'T3', u'T4'), u'T4': {'mention': u'Cisco Systems, Inc.', 'abs_char_start': 510, 'abs_char_end': 529, 'entity_type': u'Org'}, u'T2': {'mention': u'Microsoft Corp', 'abs_char_start': 313, 'abs_char_end': 327, 'entity_type': u'Org'}, u'T3': {'mention': u'Google Inc', 'abs_char_start': 429, 'abs_char_end': 439, 'entity_type': u'Org'}, u'T1': {'mention': u'Cisco System', 'abs_char_start': 281, 'abs_char_end': 293, 'entity_type': u'Org'}}}
document: Document b5dfde83-dde0-4235-b749-4c2aaf8abd06
annottions: {u'R1': (u'Contractor-Contractee', u'T1', u'T2'), u'R2': (u'Contractor-Contractee', u'T3', u'T4'), u'T4': {'mention': u'Cisco Systems, Inc.', 'abs_char_start': 510, 'abs_char_end': 529, 'entity_type': u'Org'}, u'T2': {'mention': u'Microsoft Corp', 'abs_cha

Mapped 2/2 (100%) of BRAT labels to candidates


In [52]:
# Fit the generative model
from snorkel.learning import GenerativeModel

gen_model = GenerativeModel()
gen_model.train(L_train, epochs=1000, decay=0.95, step_size=0.01 / L_train.shape[0], reg_param=1e-6)

Inferred cardinality: 2


In [20]:
# error analysis of the generative model
tp, fp, tn, fn = gen_model.error_analysis(session, L_train, L_gold_train)

# generate the noisy-aware lables (marginals) for the training data
train_marginals = gen_model.marginals(L_train)
print train_marginals

Scores (Un-adjusted)
Pos. class accuracy: 1.0
Neg. class accuracy: 0.0
Precision            1.0
Recall               1.0
F1                   1.0
----------------------------------------
TP: 2 | FP: 0 | TN: 0 | FN: 0

[ 0.98911205  0.98911205]


In [21]:
# the coverage of the generative model for the LFs
gen_model.learned_lf_stats()

,Accuracy,Coverage,Precision,Recall
0,0.907386,0.7731,0.908340,0.698566
1,0.895701,0.7699,0.891178,0.691393


In [22]:
# label dev data using brat
brat.init_collection("contract/dev", split=1, overwrite=False)
brat.view("contract/dev")

Error! Collection at 'contract/dev' already exists. Please set overwrite=True to erase all existing annotations.


In [23]:
# load up the gold labels for dev data
dev_cands = session.query(rel_contract).filter(rel_contract.split == 1).order_by(rel_contract.id).all()
brat.import_gold_labels(session, "contract/dev", dev_cands)
L_gold_dev = load_gold_labels(session, annotator_name='brat', split=1)
print L_gold_dev



documents
{u'b745346e-16a3-46ba-8adc-92b1fa085c38': Document b745346e-16a3-46ba-8adc-92b1fa085c38}

annotations
{'b745346e-16a3-46ba-8adc-92b1fa085c38': {u'R1': (u'Contractor-Contractee', u'T1', u'T2'), u'R2': (u'Contractor-Contractee', u'T3', u'T4'), u'T4': {'mention': u'The Hive Inc.', 'abs_char_start': 575, 'abs_char_end': 588, 'entity_type': u'Org'}, u'T2': {'mention': u'Cisco Systems, Inc.', 'abs_char_start': 187, 'abs_char_end': 206, 'entity_type': u'Org'}, u'T3': {'mention': u'Starbucks Corp', 'abs_char_start': 484, 'abs_char_end': 498, 'entity_type': u'Org'}, u'T1': {'mention': u'Microsoft Corporation', 'abs_char_start': 88, 'abs_char_end': 109, 'entity_type': u'Org'}}}
document: Document b745346e-16a3-46ba-8adc-92b1fa085c38
annottions: {u'R1': (u'Contractor-Contractee', u'T1', u'T2'), u'R2': (u'Contractor-Contractee', u'T3', u'T4'), u'T4': {'mention': u'The Hive Inc.', 'abs_char_start': 575, 'abs_char_end': 588, 'entity_type': u'Org'}, u'T2': {'mention': u'Cisco Systems, Inc.'

Mapped 1/2 (50%) of BRAT labels to candidates


In [24]:
# Training an LSTM model
# Carefule that the no. of relationships in dev dataset must >=2
from snorkel.learning.disc_models.rnn import reRNN

train_kwargs = {
    'lr':         0.01,
    'dim':        50,
    'n_epochs':   20,
    'dropout':    0.25,
    'print_freq': 1,
    'max_sentence_length': 100
}

lstm = reRNN(seed=1701, n_threads=None)
lstm.train(train_cands, train_marginals, X_dev=dev_cands, Y_dev=L_gold_dev, **train_kwargs)

/usr/local/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[reRNN] Training model
[reRNN] n_train=2  #epochs=20  batch size=2
[reRNN] Epoch 0 (0.20s)	Average loss=0.695944	Dev F1=66.67
[reRNN] Epoch 1 (0.36s)	Average loss=0.560884	Dev F1=66.67
[reRNN] Epoch 2 (0.51s)	Average loss=0.303893	Dev F1=66.67
[reRNN] Epoch 3 (0.64s)	Average loss=0.113205	Dev F1=66.67
[reRNN] Epoch 4 (0.77s)	Average loss=0.070595	Dev F1=66.67
[reRNN] Epoch 5 (0.89s)	Average loss=0.060619	Dev F1=66.67
[reRNN] Epoch 6 (1.02s)	Average loss=0.060886	Dev F1=66.67
[reRNN] Epoch 7 (1.15s)	Average loss=0.064195	Dev F1=66.67
[reRNN] Epoch 8 (1.29s)	Average loss=0.067799	Dev F1=66.67
[reRNN] Epoch 9 (1.42s)	Average loss=0.070830	Dev F1=66.67
[reRNN] Epoch 10 (1.54s)	Average loss=0.073072	Dev F1=66.67
[reRNN] Epoch 11 (1.67s)	Average loss=0.074511	Dev F1=66.67
[reRNN] Epoch 12 (1.80s)	Average loss=0.075199	Dev F1=66.67
[reRNN] Epoch 13 (1.92s)	Average loss=0.075198	Dev F1=66.67
[reRNN] Epoch 14 (2.05s)	Average loss=0.074575	Dev F1=66.67
[reRNN] Epoch 15 (2.17s)	Average loss=0.073

In [27]:
# The accuracy of the lstm for the dev data
p, r, f1 = lstm.score(dev_cands, L_gold_dev)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

Prec: 0.500, Recall: 1.000, F1 Score: 0.667


In [28]:
L_dev = labeler.apply_existing(split=1)
dev_marginals = gen_model.marginals(L_dev) 
lstm_dev_marginals = lstm.marginals(dev_cands)
for i in range(8):
    print L_dev.get_candidate(session, i).get_parent() # sentence
    print L_dev.get_candidate(session, i).org1.get_span(), " ", L_dev.get_candidate(session, i).org2.get_span() # candidate
    print L_dev.get_candidate(session, i).labels # labels by LFs
    print dev_marginals[i] # labels by generative models
    print lstm_dev_marginals[i]
    print 



Clearing existing...
Running UDF...
[========================================] 100%

Sentence(Document b745346e-16a3-46ba-8adc-92b1fa085c38,0,THIS PURCHASE AND SALE AGREEMENT is entered into this  23rd Feb,  2017,  by and between Microsoft Corporation (hereinafter Buyer), 600 North Wolfe Street, Baltimore, Maryland, 21287, and Cisco Systems, Inc. (hereinafter Company), 3700 Momentum St, San Jose, CA 95134-2206, USA, and here is it.)
Microsoft Corporation   Cisco Systems, Inc.
[Label (LF_between_and = 1), Label (LF_agreement = 1)]
0.989112051215
0.998094

Sentence(Document b745346e-16a3-46ba-8adc-92b1fa085c38,3,THIS PURCHASE AND SALE AGREEMENT is entered into this  11 Feb,  2020,  by and between Starbucks Corp (hereinafter Buyer), 600 North Wolfe Street, Baltimore, Maryland, 21287,and The Hive Inc. (hereinafter Company), 3700 Momentum St, San Jose, CA 95134-2206, USA.)
Starbucks Corp   Hive Inc.
[Label (LF_between_and = 1), Label (LF_agreement = 1)]
0.989112051215
0.998075



KeyError: 2

In [ ]:
# finish the labeling of  test data
brat.init_collection("contract/test", split=2, overwrite=False)
brat.view("contract/test")

In [125]:
# load up the gold labels for test data
test_cands = session.query(rel_contract).filter(rel_contract.split == 2).order_by(rel_contract.id).all()
print test_cands
brat.import_gold_labels(session, "contract/test", test_cands)
L_gold_test = load_gold_labels(session, annotator_name='brat', split=2)
print L_gold_test

[Contractor-Contractee(Span("SmartPhone Services", sentence=10, chars=[0,18], words=[0,1]), Span("Darshi Inc.", sentence=10, chars=[24,34], words=[3,4]))]
  (0, 0)	1


Mapped 1/1 (100%) of BRAT labels to candidates


In [127]:
L_test = labeler.apply_existing(split=2)
# output marginals of the generative model
# It should be noted that the generative model depends on
# the LFs and operates on the output of labeler

test_marginals = gen_model.marginals(L_test) 
lstm_test_marginals = lstm.marginals(dev_cands)
for i in range(1):
    print L_test.get_candidate(session, i).get_parent() # sentence
    print L_test.get_candidate(session, i) # candidate
    print L_dev.get_candidate(session, i).org1.get_span(), " ", L_dev.get_candidate(session, i).org2.get_span() # candidate
    print L_test.get_candidate(session, i).labels # labels by LFs
    print test_marginals[i] # labels by generative models
    print lstm_test_marginals[i]
    print 

Clearing existing...
Running UDF...
[========================================] 100%

Sentence(Document c5dbaa73-8076-4385-944c-1539a30152d7,0,SmartPhone Services and Darshi Inc. agree that the contract is valid.)
Contractor-Contractee(Span("SmartPhone Services", sentence=10, chars=[0,18], words=[0,1]), Span("Darshi Inc.", sentence=10, chars=[24,34], words=[3,4]))
Starbucks Corp   CA 95134-2206
[]
0.5
0.99784

